This notebook tries MCMC sampling of kernel hyperparameters of GP and goes for average EI in Bayesian optimization (as suggested in https://arxiv.org/pdf/1206.2944.pdf)

For simplicity considering 1D signal and 1 kernel hyperparameter (length for Gaussian kernel)

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import scipy.stats
import time
from IPython import display
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
true_y = np.random.randn(1) * np.ones(1000)
for i in range(1, 6):
    true_y += np.random.randn(1) * np.sin(i * math.pi * x)
    true_y += np.random.randn(1) * np.cos(i * math.pi * x)

## Standard machinery for GP inference

In [ ]:
def k_gaussian(x1, x2, l=0.1):
    # calculating gaussian kernel matrix
    # the output has shape (len(x2), len(x1))
    # entry at [i, j] position is given by k(x2[i], x1[j])
    # dealing with gaussian kernels so k(x, y) = e ^ ((x - y) ** 2 / 2 * l ** 2)
    
    # gaussian kernel hyperparameters - adjusts the distance between points and variance
    scale_kernel = 1
    
    x1_matrix = np.tile(x1, len(x2)).reshape((len(x2), len(x1)))
    x2_matrix = np.tile(x2, len(x1)).reshape((len(x1), len(x2))).transpose()
    
    k_matrix = np.exp(-(x1_matrix - x2_matrix) ** 2 / (2 * l * l)) * scale_kernel ** 2
    
    return k_matrix

In [ ]:
def log_likelihood(sample_x, sample_y, sigma_obs, l):
    # The following function calculates the log-likelihood of observed data
    # wrt to prior distribution for GP, i.e. zero mean and sigma given by k_gaussian(l) + sigma_obs ** 2 * I
    
    # Under that model the log-likelihood is given by 
    # -0.5 * y' * sigma(-1) * y - 0.5 * n * log(2pi) - 0.5 * log|sigma|
    
    # To make sense of the code below note that we express log-likelihood through the cholesky decomposition of sigma
    # Then |sigma| = |chol| ** 2 (det of product is product of det)
    # |chol| = prod(chol_ii) (because cholesky matrix is lower triangular)
    # Thus, 0.5 * log|sigma| = sum(log(chol_ii))
    
    sigma = k_gaussian(sample_x, sample_x, l) + np.eye(len(sample_x)) * sigma_obs ** 2
    chol = np.linalg.cholesky(sigma)
    
    # Calculating alpha = sigma(-1) * y (or solution to sigma * alpha = y) using cholesky matrix
    # (This trick is taken from sklearn implementation of GP)
    alpha = sp.linalg.cho_solve((chol, True), sample_y).reshape((-1, 1))
    
    log_lik = -0.5 * np.dot(sample_y.reshape(1, -1), alpha)
    log_lik -= 0.5 * len(sample_x) * np.log(2 * np.pi)
    log_lik -= np.trace(np.log(np.absolute(chol)))
    
    return log_lik[0][0]

In [ ]:
def gp_posterior(sample_x, sample_y, x, sigma_obs, l):
    # Calculating posterior for gaussian processes
    # I am specifically interested in posterior mean, std and cholesky matrix for postrior at sampled points (for nei)
    # it is assumed that observations have some additional gaussian noise
    
    # Important: the method cannot handle sigma_obs=0 if I want to predict for sample_x
    # Mostly numerical issues: with zero noise matrix to invert may not be positive-semidefinite
    
    # Separately calculating matrix used to calculate both mean and variance
    K = np.dot(k_gaussian(sample_x, x, l),
               np.linalg.inv(k_gaussian(sample_x, sample_x, l) + np.eye(len(sample_x)) * sigma_obs ** 2)
              )
    
    mu = np.dot(K, sample_y)
    sigma = k_gaussian(x, x, l) - np.dot(K, k_gaussian(x, sample_x, l))
    std_1d = np.sqrt([sigma[i, i] for i in range(len(mu))])
    
    return mu.reshape(-1), std_1d.reshape(-1)

## MCMC section

In [ ]:
# Picking a uniform [0, 1] prior for the kernel hyperparameter (10 ** (-8) is added for more stability during MCMC)
l_prior = lambda l: int(l > 0 and l < 1) + 10 ** (-8) * int(l <= 0 or l >= 1)
# For simplicity specifying inverse cdf as well
l_icdf = lambda l: l
# Specifying jumping distribution - has to be symmetric to be viable for several MCMC algos (e.g. Metropolis-Hastings)
l_jump = lambda l: l + 0.05 * np.random.randn()

In [ ]:
def MCMC_l_sampler(sample_x, sample_y, sigma_obs, 
                   l_prior, l_icdf, l_jump,
                   burn_period, mcmc_samples):
    
    # This function performs Bayesian MCMC sampling for Gaussian kernel hyperparameters
    # Specifically, the first point is sampled using inverse cdf for l_prior
    # Moves are suggested using l_jump function
    # Moves are accepted / rejected with Metropolis-Hastings algorithms
    # (i.e. true posterior density is proportional to exp(log_likelihood) * l_prior, 
    # ratio of posterior values give the probability of acception a move)
    # First burn_period samples of l are discarded and n_samples consecutive samples are the output of a function
    
    # MCMC is concerned with the ratio of true probabilities
    # However, for efficiency reasons we express everything through log-lokelihoods
    log_posterior = lambda l: log_likelihood(sample_x, sample_y, sigma_obs, l) + np.log(l_prior(l))
    
    l = l_icdf(np.random.rand())
    past_log_posterior = log_posterior(l)
    for _ in range(burn_period):
        next_l = l_jump(l)
        next_log_posterior = log_posterior(next_l)
        if np.log(np.random.randn()) < (next_log_posterior - past_log_posterior):
            l = next_l
            past_log_posterior = next_log_posterior
            
    sampled_l = []
    for _ in range(mcmc_samples):
        next_l = l_jump(l)
        next_log_posterior = log_posterior(next_l)
        if np.log(np.random.randn()) < (next_log_posterior - past_log_posterior):
            l = next_l
            past_log_posterior = next_log_posterior
        sampled_l.append(l)

    return sampled_l

In [ ]:
sigma_obs = 0.5

pick_x = np.random.choice(range(len(x)), size=5, replace=False)
sample_x = x[pick_x]
sample_y = true_y[pick_x] + sigma_obs * np.random.randn(5)

sampled_l = MCMC_l_sampler(sample_x, sample_y, sigma_obs, 
                           l_prior, l_icdf, l_jump,
                           burn_period=10000, mcmc_samples=1000)

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(np.linspace(0.01,1,100), 
         [np.exp(log_likelihood(sample_x, sample_y, sigma_obs, l)) for l in np.linspace(0.01,1,100)])
plt.title("True density of posterior distribution (not scaled)")
plt.xlabel("l")
plt.ylabel("likelihood")
plt.show()

plt.subplot(2, 1, 1)
plt.hist(sampled_l, bins=25, normed=1)
plt.title("Density of MCMC sampled kernel hyperparameters")
plt.xlim([0, 1])
plt.xlabel("l")
plt.ylabel("Normalized frequency")
plt.show()

## EI optimization section

In [ ]:
sigma_obs = 0.5
burn_period = 10000
mcmc_samples = 25
l_prior = lambda l: int(l > 0 and l < 1) + 10 ** (-8) * int(l <= 0 or l >= 1)
l_icdf = lambda l: l
l_jump = lambda l: l + 0.05 * np.random.randn()


pick_x = np.random.choice(range(len(x)), size=3, replace=False)
sample_x = x[pick_x]
sample_y = true_y[pick_x] + sigma_obs * np.random.randn(3)
best_y = np.max(sample_y)

for t in range(1, 100):
    
    # Sampling possible kernel hyprparameters
    sampled_l = MCMC_l_sampler(sample_x, sample_y, sigma_obs, 
                               l_prior, l_icdf, l_jump,
                               burn_period, mcmc_samples)
    
    # Averaging GP posterior and EI over possible kernel hyperparameters
    # Note that as std is not quite an expectation, its averaging is a hack and not necessariy would give true std
    mu = np.zeros((len(x),))
    std_1d = np.zeros((len(x),))
    ei = np.zeros((len(x),))
    for l in sampled_l:
        sampled_mu, sampled_std_1d = gp_posterior(sample_x, sample_y, x, sigma_obs, l)

        z = (sampled_mu - best_y) / sampled_std_1d
        sampled_ei = sampled_std_1d * scipy.stats.norm.pdf(z) + z * sampled_std_1d * scipy.stats.norm.cdf(z)
        
        mu += sampled_mu
        std_1d += sampled_std_1d
        ei += sampled_ei
    

    pick_x = np.argmax(ei)
    sample_x = np.append(sample_x, x[pick_x])
    sample_y = np.append(sample_y, true_y[pick_x] + sigma_obs * np.random.randn(1))
    best_y = max(sample_y[-1], best_y)
    
    mu /= mcmc_samples
    std_1d /= mcmc_samples
        
    plt.plot(x, true_y, label="true_signal")
    plt.plot(sample_x, sample_y, ".", color="r", label="picked_x")
    plt.plot([sample_x[-1]], [sample_y[-1]], ".", color="b", label="last_x")
    plt.plot(x, mu, color="g", label="posterior")
    plt.fill_between(x, mu - 2 * std_1d, mu + 2 * std_1d, color="g", alpha=0.5)

    plt.title("True and recovered signals")
    plt.legend()
    plt.xlabel("x")
    plt.savefig("images/mcmc_iter_{}.png".format(t))
    plt.show()

    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(2)
    

In [ ]:
import imageio
images = []
filenames = ["images/mcmc_iter_{}.png".format(t) for t in range(1, 80)]
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('images/mcmc_opt.gif', images)